In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/embfts")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from models.MimoNonStationaryFtsPca import MimoNonStationaryFtsPca
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math


np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
def sample_first_prows(data, perc):
    return data.head(int(len(data)*(perc)))

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [5]:
# convert series to supervised learning
def series_to_supervised_mimo(data, n_in, n_out, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [(df.columns[j]+'(t)') for j in range(n_vars)]
		else:
			names += [(df.columns[j]+'%d(t+%d)' % (j, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [6]:
filepath = '/home/hugo/projetos-doutorado/mimo_emb_fts/data/household_power_consumption.csv'
data = pd.read_csv(filepath, sep = ";")
data = data.drop(labels=['Time','Date'], axis=1)
data = data.loc[0: : 30]
data.dropna(inplace = True)
data = clean_dataset(data)
data.head()


/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,4.216,0.418,234.84,18.4,0.0,1.0,17.0
30,2.720,0.000,235.06,11.6,0.0,0.0,17.0
60,3.452,0.000,235.20,15.2,0.0,1.0,17.0
90,4.298,0.000,232.39,18.4,0.0,1.0,16.0
120,3.262,0.052,232.64,14.0,0.0,0.0,17.0


In [7]:
data['Global_active_power'] = pd.to_numeric(data['Global_active_power'],errors='coerce')
data['Global_reactive_power'] = pd.to_numeric(data['Global_reactive_power'],errors='coerce')
data['Voltage'] = pd.to_numeric(data['Voltage'],errors='coerce')
data['Global_intensity'] = pd.to_numeric(data['Global_intensity'],errors='coerce')
data['Sub_metering_1'] = pd.to_numeric(data['Sub_metering_1'],errors='coerce')
data['Sub_metering_2'] = pd.to_numeric(data['Sub_metering_2'],errors='coerce')
data['Sub_metering_3'] = pd.to_numeric(data['Sub_metering_3'],errors='coerce')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68308 entries, 0 to 2075250
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Global_active_power    68308 non-null  float64
 1   Global_reactive_power  68308 non-null  float64
 2   Voltage                68308 non-null  float64
 3   Global_intensity       68308 non-null  float64
 4   Sub_metering_1         68308 non-null  float64
 5   Sub_metering_2         68308 non-null  float64
 6   Sub_metering_3         68308 non-null  float64
dtypes: float64(7)
memory usage: 4.2 MB


In [8]:
# frame as supervised learning
reframed = series_to_supervised_mimo(data, 1, 1)
reframed.head()

,Global_active_power(t-1),Global_reactive_power(t-1),Voltage(t-1),Global_intensity(t-1),Sub_metering_1(t-1),Sub_metering_2(t-1),Sub_metering_3(t-1),Global_active_power(t),Global_reactive_power(t),Voltage(t),Global_intensity(t),Sub_metering_1(t),Sub_metering_2(t),Sub_metering_3(t)
30,4.216,0.418,234.84,18.4,0.0,1.0,17.0,2.720,0.000,235.06,11.6,0.0,0.0,17.0
60,2.720,0.000,235.06,11.6,0.0,0.0,17.0,3.452,0.000,235.20,15.2,0.0,1.0,17.0
90,3.452,0.000,235.20,15.2,0.0,1.0,17.0,4.298,0.000,232.39,18.4,0.0,1.0,16.0
120,4.298,0.000,232.39,18.4,0.0,1.0,16.0,3.262,0.052,232.64,14.0,0.0,0.0,17.0
150,3.262,0.052,232.64,14.0,0.0,0.0,17.0,3.214,0.078,232.64,13.8,0.0,0.0,17.0


In [9]:
cols = reframed.columns
cols

Index(['Global_active_power(t-1)', 'Global_reactive_power(t-1)',
       'Voltage(t-1)', 'Global_intensity(t-1)', 'Sub_metering_1(t-1)',
       'Sub_metering_2(t-1)', 'Sub_metering_3(t-1)', 'Global_active_power(t)',
       'Global_reactive_power(t)', 'Voltage(t)', 'Global_intensity(t)',
       'Sub_metering_1(t)', 'Sub_metering_2(t)', 'Sub_metering_3(t)'],
      dtype='object')

In [10]:
df_forecats_columns = [ 'Global_active_power(t)',
       'Global_reactive_power(t)', 'Voltage(t)', 'Global_intensity(t)',
       'Sub_metering_1(t)', 'Sub_metering_2(t)', 'Sub_metering_3(t)']

In [11]:
def sliding_window_pca(data,n_windows,train_size,steps_ahead,transformation):

    result = {
         "window": [],
         "rmse": [],
         "nrmse": [],
         "mae": [],
         "r2": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "nrmse": [],
         "mae": [],
         "r2": [],
         "variable":[]
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            df_train = ttrain.loc[:,'Global_active_power(t-1)':'Sub_metering_3(t-1)']
            df_test = ttest.loc[:,'Global_active_power(t-1)':'Sub_metering_3(t-1)']
            df_original = ttest.loc[:,'Global_active_power(t)':'Sub_metering_3(t)']
            
            models, data_train = mimo_kpca_nsfts.run_train(df_train,transformation)
            forecast, data_test = mimo_kpca_nsfts.run_test(models,df_test,steps_ahead,transformation)
            
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
                original = original[:len(original)-1]
                forecast = forecast[1:]
                
                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = mean_absolute_error(original,forecast)
                r2 = r2_score(original,forecast)
                rmse = mean_squared_error(original,forecast,squared=False)
                nrmse = cal_nrmse(rmse, original)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

## Sliding Window: MIMO KPCA-NSFTS

In [42]:
num_components_pca = 2
order_fts_model = 1
npart = 70
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [43]:
kpca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='KPCA')

--------------------
training window 0
--------------------
training window 2276
--------------------
training window 4552
--------------------
training window 6828
--------------------
training window 9104
--------------------
training window 11380
--------------------
training window 13656
--------------------
training window 15932
--------------------
training window 18208
--------------------
training window 20484
--------------------
training window 22760
--------------------
training window 25036
--------------------
training window 27312
--------------------
training window 29588
--------------------
training window 31864
--------------------
training window 34140
--------------------
training window 36416
--------------------
training window 38692
--------------------
training window 40968
--------------------
training window 43244
--------------------
training window 45520
--------------------
training window 47796
--------------------
training window 50072
-------------------

In [46]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "nrmse": [],
    "mae": [],
    "r2": []
}

measures = kpca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = statistics.mean(var_agr.loc[:,'rmse'])
    nrmse = statistics.mean(var_agr.loc[:,'nrmse'])
    mae = statistics.mean(var_agr.loc[:,'mae'])
    r2 = statistics.mean(var_agr.loc[:,'r2'])

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 



In [47]:
print("Statistics MIMO KPCA-NSFTS(test): ")
final_measures_kpca

Statistics MIMO KPCA-NSFTS(test): 


,variable,rmse,nrmse,mae,r2
0,Global_active_power(t),0.442800,0.074060,0.361774,0.787108
1,Global_reactive_power(t),0.063589,0.100301,0.052929,0.640939
2,Voltage(t),0.961236,0.059425,0.791506,0.874309
3,Global_intensity(t),1.881605,0.073923,1.537342,0.781696
4,Sub_metering_1(t),1.168499,NaN,0.963582,0.960725
5,Sub_metering_2(t),1.134705,0.042455,0.936345,0.918442
6,Sub_metering_3(t),3.860249,0.165865,3.164840,0.770527


## Sliding Window: MIMO PCA-NSFTS

In [48]:
num_components_pca = 4
order_fts_model = 1
npart = 70
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [49]:
pca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='PCA')

--------------------
training window 0
--------------------
training window 2276
--------------------
training window 4552
--------------------
training window 6828
--------------------
training window 9104
--------------------
training window 11380
--------------------
training window 13656
--------------------
training window 15932
--------------------
training window 18208
--------------------
training window 20484
--------------------
training window 22760
--------------------
training window 25036
--------------------
training window 27312
--------------------
training window 29588
--------------------
training window 31864
--------------------
training window 34140
--------------------
training window 36416
--------------------
training window 38692
--------------------
training window 40968
--------------------
training window 43244
--------------------
training window 45520
--------------------
training window 47796
--------------------
training window 50072
-------------------

In [50]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "nrmse": [],
    "mae": [],
    "r2": []
}

measures = pca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = statistics.mean(var_agr.loc[:,'rmse'])
    nrmse = statistics.mean(var_agr.loc[:,'nrmse'])
    mae = statistics.mean(var_agr.loc[:,'mae'])
    r2 = statistics.mean(var_agr.loc[:,'r2'])

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics MIMO PCA-NSFTS(test): ")
final_measures_pca

Statistics MIMO PCA-NSFTS(test): 


,variable,rmse,nrmse,mae,r2
0,Global_active_power(t),0.906885,0.154592,0.697612,0.055537
1,Global_reactive_power(t),0.138311,0.217115,0.109245,-0.681850
2,Voltage(t),2.782284,0.181175,2.214527,-0.258356
3,Global_intensity(t),3.827782,0.152371,2.933834,0.057735
4,Sub_metering_1(t),7.256558,inf,5.157115,-0.507716
5,Sub_metering_2(t),6.010053,0.159982,4.280566,-0.410048
6,Sub_metering_3(t),9.199294,0.394131,7.458465,-0.356427
